In [1]:
!nvidia-smi

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Tue Feb 21 16:31:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--------------

In [2]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
import keras
import keras.backend as K
from keras.layers import Conv2D, BatchNormalization, Activation, Dense, Input, Add, GlobalAveragePooling2D, UpSampling2D, Concatenate
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
import matplotlib.pyplot as plt

In [3]:
print(tf.__version__)       
print(keras.__version__)    

2.9.2
2.9.0


In [13]:
train_datagen = ImageDataGenerator(rescale = 1/255.0,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   brightness_range = (0.1, 0.25),
                                   rotation_range = 20,
                                   vertical_flip = True,
                                   horizontal_flip = True,
                                   validation_split = 0.2)
 
# test_datagen =  ImageDataGenerator(rescale = 1/255.0,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    brightness_range = (0.1, 0.25),
#                                    rotation_range = 20,
#                                    vertical_flip = True,
#                                    horizontal_flip = True,)
    
    
training_set = train_datagen.flow_from_directory('/kaggle/input/imagenet11/ImageNet-13 Original Image Dateset/image',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical',
                                                 subset = 'training')

valid_set = train_datagen.flow_from_directory('/kaggle/input/imagenet11/ImageNet-13 Original Image Dateset/image',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical',
                                            subset = 'validation'
                                            )

# test_set = test_datagen.flow_from_directory('/kaggle/input/dogs-cats-images/dataset/test_set/',
#                                             target_size = (256, 256),
#                                             batch_size = 16,
#                                             class_mode = 'categorical',)

Found 13400 images belonging to 13 classes.
Found 3350 images belonging to 13 classes.


In [17]:
def Conv_Block(inputs, n_filters):

  y = Conv2D(filters=n_filters, kernel_size=3, strides=1, padding='same', use_bias=False)(inputs)
  y = BatchNormalization()(y)
  y = Activation('relu')(y)

  return y

def Strided_Conv_Block(inputs, n_filters, n_stride):

  y = Conv2D(filters=n_filters, kernel_size=3, strides=2, padding='same', use_bias=False)(inputs)
  y = BatchNormalization()(y)
  y = Activation('relu')(y)

  return y

def Multi_Resolution_Parallel_Convolution(inputs, n_filters):

  y = Conv_Block(inputs, n_filters)
  y = Conv_Block(y, n_filters)
  y = Conv_Block(y, n_filters)
  y = Conv_Block(y, n_filters)

  return y

def Multi_Resolution_Fusion1(stage1_in, stage2_in, stage3_in, n_filters):

  stage2_i = Strided_Conv_Block(stage1_in, 2*n_filters, n_stride=2)
  stage2_ii = Strided_Conv_Block(stage2_i, 4*n_filters, n_stride=2)
  stage3 = Concatenate()([stage3_in, stage2_ii])
  stage3 = Conv2D(filters=4*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage3)
  stage3 = BatchNormalization()(stage3)
  stage3 = Activation('relu')(stage3)


  stage2 = Concatenate()([stage2_in, stage2_i])
  stage2 = Conv2D(filters=2*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage2)
  stage2 = BatchNormalization()(stage2)
  stage2 = Activation('relu')(stage2)


  stage1_i = UpSampling2D(size=2, interpolation='bilinear')(stage2_in)
  stage1 = Concatenate()([stage1_in, stage1_i])
  stage1 = Conv2D(filters=n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage1)
  stage1 = BatchNormalization()(stage1)
  stage1 = Activation('relu')(stage1)

  return stage1, stage2, stage3

def Multi_Resolution_Fusion2(stage1_in, stage2_in, stage3_in, stage4_in, n_filters):

  stage4_i = Strided_Conv_Block(stage1_in, 2*n_filters, n_stride=2)
  stage4_ii = Strided_Conv_Block(stage4_i, 4*n_filters, n_stride=2)
  stage4_iii = Strided_Conv_Block(stage4_ii, 8*n_filters, n_stride=2)

  stage4_iii_i = Strided_Conv_Block(stage2_in, 4*n_filters, n_stride=2)
  stage4_iii_ii = Strided_Conv_Block(stage4_iii_i, 8*n_filters, n_stride=2)

  stage4 = Concatenate()([stage4_in, stage4_iii, stage4_iii_ii])
  stage4 = Conv2D(filters=8*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage4)
  stage4 = BatchNormalization()(stage4)
  stage4 = Activation('relu')(stage4)


  stage3 = Concatenate()([stage3_in, stage4_ii, stage4_iii_i])
  stage3 = Conv2D(filters=4*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage3)
  stage3 = BatchNormalization()(stage3)
  stage3 = Activation('relu')(stage3)


  stage2_i = UpSampling2D(size=2, interpolation='bilinear')(stage3_in)
  stage2 = Concatenate()([stage2_in, stage4_i, stage2_i])
  stage2 = Conv2D(filters=2*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage2)
  stage2 = BatchNormalization()(stage2)
  stage2 = Activation('relu')(stage2)


  stage1_i = UpSampling2D(size=2, interpolation='bilinear')(stage2_in)
  stage1_ii = UpSampling2D(size=4, interpolation='bilinear')(stage3_in)
  stage1 = Concatenate()([stage1_in, stage1_i, stage1_ii])
  stage1 = Conv2D(filters=n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage1)
  stage1 = BatchNormalization()(stage1)
  stage1 = Activation('relu')(stage1)

  return stage1, stage2, stage3, stage4

def Multi_Resolution_Fusion3(stage1_in, stage2_in, stage3_in, stage4_in, n_filters):

  stage4_i = Strided_Conv_Block(stage1_in, 2*n_filters, n_stride=2)
  stage4_ii = Strided_Conv_Block(stage4_i, 4*n_filters, n_stride=2)
  stage4_iii = Strided_Conv_Block(stage4_ii, 8*n_filters, n_stride=2)

  stage4_iii_i = Strided_Conv_Block(stage2_in, 4*n_filters, n_stride=2)
  stage4_iii_ii = Strided_Conv_Block(stage4_iii_i, 8*n_filters, n_stride=2)

  stage4_iii_ii_i = Strided_Conv_Block(stage3_in, 8*n_filters, n_stride=2)

  stage4 = Concatenate()([stage4_in, stage4_iii, stage4_iii_ii, stage4_iii_ii_i])
  stage4 = Conv2D(filters=8*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage4)
  stage4 = BatchNormalization()(stage4)
  stage4 = Activation('relu')(stage4)


  stage3_i = UpSampling2D(size=2, interpolation='bilinear')(stage4_in)
  stage3 = Concatenate()([stage3_in, stage4_ii, stage4_iii_i, stage3_i])
  stage3 = Conv2D(filters=4*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage3)
  stage3 = BatchNormalization()(stage3)
  stage3 = Activation('relu')(stage3)


  stage2_i = UpSampling2D(size=2, interpolation='bilinear')(stage3_in)
  stage2_ii = UpSampling2D(size=4, interpolation='bilinear')(stage4_in)
  stage2 = Concatenate()([stage2_in, stage4_i, stage2_i, stage2_ii])
  stage2 = Conv2D(filters=2*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage2)
  stage2 = BatchNormalization()(stage2)
  stage2 = Activation('relu')(stage2)


  stage1_i = UpSampling2D(size=2, interpolation='bilinear')(stage2_in)
  stage1_ii = UpSampling2D(size=4, interpolation='bilinear')(stage3_in)
  stage1_iii = UpSampling2D(size=8, interpolation='bilinear')(stage4_in)
  stage1 = Concatenate()([stage1_in, stage1_i, stage1_ii, stage1_iii])
  stage1 = Conv2D(filters=n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage1)
  stage1 = BatchNormalization()(stage1)
  stage1 = Activation('relu')(stage1)

  return stage1, stage2, stage3, stage4

def HRNet(inputs, n_filters):

  # inputs = Input(input_shape)

  stage1 = Multi_Resolution_Parallel_Convolution(inputs, n_filters)
  stage2 = Strided_Conv_Block(stage1, 2*n_filters, n_stride=2)

  stage1 = Conv_Block(stage1, n_filters)
  stage1 = Multi_Resolution_Parallel_Convolution(stage1, n_filters) 
  stage2 = Multi_Resolution_Parallel_Convolution(stage2, 2*n_filters)
  stage3 = Strided_Conv_Block(stage2, 4*n_filters, n_stride=2)

  stage1, stage2, stage3 = Multi_Resolution_Fusion1(stage1, stage2, stage3, n_filters)
  stage1 = Multi_Resolution_Parallel_Convolution(stage1, n_filters)
  stage2 = Multi_Resolution_Parallel_Convolution(stage2, 2*n_filters)
  stage3 = Multi_Resolution_Parallel_Convolution(stage3, 4*n_filters)
  stage4 = Strided_Conv_Block(stage3, 8*n_filters, n_stride=2)

  stage1, stage2, stage3, stage4 = Multi_Resolution_Fusion2(stage1, stage2, stage3, stage4, n_filters)
  stage1 = Multi_Resolution_Parallel_Convolution(stage1, n_filters)
  stage2 = Multi_Resolution_Parallel_Convolution(stage2, 2*n_filters)
  stage3 = Multi_Resolution_Parallel_Convolution(stage3, 4*n_filters)
  stage4 = Multi_Resolution_Parallel_Convolution(stage4, 8*n_filters)
  stage1, stage2, stage3, stage4 = Multi_Resolution_Fusion3(stage1, stage2, stage3, stage4, n_filters)
  # print(stage1.shape, stage2.shape, stage3.shape, stage4.shape)
  
  return stage1, stage2, stage3, stage4

def HRNet_Classification(input_shape, n_classes, n_filters):

  inputs = Input(input_shape)

  stage1, stage2, stage3, stage4 = HRNet(inputs, n_filters=n_filters)

  stage1 = Conv2D(filters=4*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage1)
  stage1 = BatchNormalization()(stage1)
  stage1 = Activation('relu')(stage1)

  stage2 = Conv2D(filters=8*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage2)
  stage2 = BatchNormalization()(stage2)
  stage2 = Activation('relu')(stage2)

  stage3 = Conv2D(filters=16*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage3)
  stage3 = BatchNormalization()(stage3)
  stage3 = Activation('relu')(stage3)

  stage4 = Conv2D(filters=32*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage4)
  stage4 = BatchNormalization()(stage4)
  stage4 = Activation('relu')(stage4)

  y1 = Strided_Conv_Block(stage1, 8*n_filters, n_stride=2)
  y = Add()([stage2, y1])

  y2 = Strided_Conv_Block(y1, 16*n_filters, n_stride=2)
  y = Add()([stage3, y2])

  y3 = Strided_Conv_Block(y2, 32*n_filters, n_stride=2)
  y = Add()([stage4, y3])

  y = Conv2D(filters=64*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(y)
  y = BatchNormalization()(y)
  y = Activation('relu')(y)

  y = Conv2D(filters=64*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(y)
  y = BatchNormalization()(y)
  y = Activation('relu')(y)

  y = GlobalAveragePooling2D()(y)
  y = Dense(n_classes, use_bias=False)(y)

  if n_classes == 1:
    y = Activation('sigmoid')(y)
  else:
    y = Activation('softmax')(y)
  # print(y.shape)

  model = Model(inputs=inputs, outputs=y)

  return model


model = HRNet_Classification(input_shape=(224,224,3), n_classes=13, n_filters=16)
# model.summary()  

In [18]:
""" callbacks """
checkpoint_filepath = './HRNet_{epoch}.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    # monitor='val_iou_score',
    # mode='max',
    verbose = 1,
    period = 3,
    save_best_only=False)

csv_path = "./SQNet.csv"
callbacks = [
    model_checkpoint_callback,
    ReduceLROnPlateau(monitor="val_loss", patience=5, factor=0.5, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor="val_loss", patience=12)]

In [ ]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),    #label_smoothing=0.1),
    optimizer=tf.keras.optimizers.Adam(0.01),
    metrics=['accuracy'])

h1 = model.fit(
  training_set,
  validation_data=valid_set,
  batch_size=16,
  epochs=1000,
  steps_per_epoch=len(training_set)//10,
  workers=-1,
  use_multiprocessing=True,
  validation_steps=len(valid_set)//10,
  callbacks=callbacks)

model.save('./HRNet_last.h5')

Epoch 1/1000
83/83 [==============================] - ETA: 0s - loss: 2.2269 - accuracy: 0.2319
Epoch 1: saving model to ./HRNet_1.h5
83/83 [==============================] - 189s 2s/step - loss: 2.2269 - accuracy: 0.2319 - val_loss: 111.3967 - val_accuracy: 0.0881 - lr: 0.0100
Epoch 2/1000
83/83 [==============================] - 174s 2s/step - loss: 2.1327 - accuracy: 0.2644 - val_loss: 2.9857 - val_accuracy: 0.1570 - lr: 0.0100
Epoch 3/1000
83/83 [==============================] - 174s 2s/step - loss: 2.1140 - accuracy: 0.2643 - val_loss: 50.5115 - val_accuracy: 0.0872 - lr: 0.0100
Epoch 4/1000
83/83 [==============================] - ETA: 0s - loss: 2.1250 - accuracy: 0.2718
Epoch 4: saving model to ./HRNet_4.h5
83/83 [==============================] - 176s 2s/step - loss: 2.1250 - accuracy: 0.2718 - val_loss: 93.2836 - val_accuracy: 0.0872 - lr: 0.0100
Epoch 5/1000
83/83 [==============================] - 175s 2s/step - loss: 2.1040 - accuracy: 0.2620 - val_loss: 42.4750 - val_acc

In [ ]:
model.evaluate_generator(test_set, steps=64, workers=-1, use_multiprocessing=True)

In [ ]:
# loss
plt.plot(h1.history['loss'], label='train loss')
plt.plot(h1.history['val_loss'], label='val loss')
plt.legend()
plt.show()

# accuracies
plt.plot(h1.history['accuracy'], label='train acc')
plt.plot(h1.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()